In [ ]:
from math import pi
import numpy as np
from pde import plot_magnitudes,PDE,movie_scalar,UnitGrid,plot_magnitudes, CartesianGrid, MemoryStorage, ScalarField, plot_kymograph, plot_kymographs, FieldCollection, PlotTracker, PDEBase, ExplicitSolver, FileStorage,VectorField
import pandas as pd
import os,sys
import matplotlib.animation as animation
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from pde.tools.numba import jit
import itertools
import numba as nb
import pickle as pickle

In [ ]:
class PDE_2D_odd(PDEBase):
    def __init__(self, alpha=1, Nx=100,Ny=100, Lx=20,Ly=20):
        #initialize parameters
        self.alpha = alpha
        self.Nx = Nx
        self.Ny = Ny
        self.Lx = Lx
        self.Ly = Ly
        
    def get_initial_state(self,u,v):
        Nx = self.Nx
        Ny =self.Ny 
        Lx =  self.Lx 
        Ly = self.Ly
        bounds=[(-Lx/2,Lx/2),(-Ly/2,Ly/2)]
        shape=[Nx,Ny]
        grid = CartesianGrid(bounds,shape, periodic=[True,True])
        return FieldCollection([ScalarField(grid,u,dtype=complex),ScalarField(grid,v,dtype=complex)])
    
#         FieldCollection([ScalarField(grid, array_u,dtype=complex),ScalarField(grid, array_v,dtype=complex)])
# 
#         # grid = UnitGrid([self.Nx,self.Nx],periodic=[True,True])
#         #random initial condition:
#         S = amp*(np.random.rand(self.Nx,self.Ny)-0.5 + 1j* (np.random.rand(self.Nx,self.Ny)-0.5))
# 
        # def multi(amp,mesh,Nx,Ny,nx,ny):
        #     return amp* (np.cos(2*np.pi *nx*mesh[0]/Nx) + 1j*np.cos(2*np.pi*ny*mesh[1]/Ny))
# 
        # S = multi(amp,np.meshgrid(np.arange(self.Ny),np.arange(self.Nx)),Nx,Ny,4,4)
        # Sdot = -1j*alpha*S# np.zeros_like(S)
#         state = FieldCollection([ScalarField(grid,S,dtype=complex),ScalarField(grid,Sdot,dtype=complex)])
#         return state
        
    def evolution_rate(self, state, t=0):
        # u=phi and v=dot(phi) 
        u, v = state
        rhs = state.copy()
        ##### Jack's nonlinear PDE, written as two first order time derivative eqs. #####
        rhs[0] = v
        rhs[1] = (u+1j*self.alpha*u + v + np.abs(u)**2*u).laplace(bc='natural')
        return rhs

    def _make_pde_rhs_numba(self, state):
        
        #### This part is to speed up things ####
        """ numba-compiled implementation of the PDE """
        alpha = self.alpha
        laplace = state.grid.get_operator("laplace", bc = "natural")
        gradient = state.grid.get_operator("gradient", bc = "natural")

        @nb.jit
        def pde_rhs(state_data, t):
            u = state_data[0]
            v = state_data[1]
            rate = np.empty_like(state_data)
            rate[0] = v
            rate[1] = laplace(u+1j*alpha*u+v+np.abs(u)**2*u)
            return rate
        return pde_rhs

In [ ]:
def ini_sine(alpha,Nx,Ny,nx,ny):
    mesh = np.meshgrid(np.arange(Ny),np.arange(Nx))
    ampx = np.sqrt((alpha*Lx/(2*nx*np.pi))**2-1)
    ampy = np.sqrt((alpha*Ly/(2*ny*np.pi))**2-1)
    u =  ampx * np.cos(2*np.pi *nx*mesh[0]/Nx) +ampy* 1j*np.cos(2*np.pi*ny*mesh[1]/Ny)
    v = -1j*alpha*u
    #constrain initial energy and momentum to be zero:
    u = u - np.average(u)
    v = v - np.average(v)    
    return u,v

def ini_ran(amp,Nx,Ny):
    u = amp*(np.random.rand(Nx,Ny)-0.5 + 1j* (np.random.rand(Nx,Ny)-0.5))
    v = amp*(np.random.rand(Nx,Ny)-0.5 + 1j* (np.random.rand(Nx,Ny)-0.5))
    #constrain initial energy and momentum to be zero:
    u = u - np.average(u)
    v = v - np.average(v)    
    return u,v

def get_initial_fromdata(file,pert=False,filter=False,t=-1):
    'generates an I.C. from the last timestep of a dataset'
    with open(file, 'rb') as output:
        dic = pickle.load(output)
        phi = dic['phi']
        dphi = dic['dphi']
        alpha = dic['alpha']
        Lx = dic['Lx']
        Ly = dic['Lx']
        Nx= dic['Nx']
        Ny= dic['Ny']
        
    def perturb(u,v):
        unoise =(np.random.rand(Nx,Ny) -0.5 ) + 1j* (np.random.rand(Nx,Ny)-0.5)
        vnoise =(np.random.rand(Nx,Ny) -0.5 ) + 1j* (np.random.rand(Nx,Ny)-0.5)
        u += 2*unoise
        v += 2*vnoise
        return u,v
    
    def amplify(u,v):
        u = u*1.5
        v = v*1.5
        return u,v
    if pert:
        u,v= perturb(u,v)
    if filter:
        u,v = amplify(u,v)
    return u[t,:,:],v[t,:,:]

In [ ]:
def sim2Dodd(alpha,Nx,Ny,Lx,Ly,pt,dt,ic):
    u,v = ic
    fields = []
    storage = MemoryStorage()
    trackers = ['progress'    , 'consistency'  ,     storage.tracker(interval=pt) ]
    eq = PDE_2D_odd(alpha=alpha,Nx=Nx,Ny=Ny,Lx=Lx,Ly=Ly)
    state = eq.get_initial_state(u, v) #initial field for u,v
    sol = eq.solve(state, t_range=tf,tracker=trackers,dt=dt)
    field =[]
    for j,i in storage.items():
        field.append(np.array(i.data))
    field=np.asarray(field)
    phi = field[:,0,:,:]
    dphi = field[:,1,:,:]
    return phi,dphi

In [ ]:
#parameters
tf =3
pt = 0.05
dt = 1e-4
alpha=5
Lx=50
Ly=50
Nx=50
Ny=50
savefolder = 'data/'
savefile = 'test'
nx=4
ny=4
def runsim(alpha,Nx,Ny,Lx,Ly):
    u,v = ini_sine(alpha,Nx,Ny,nx,ny)
    phi,dphi = sim2Dodd(alpha,Nx,Ny,Lx,Ly,pt,dt,[u,v])
    filename = savefolder+savefile
    with open(filename, 'wb') as output:
        print('saved as:   ', filename)
        dic = {'phi':phi,'dphi':dphi,'Lx':Lx,'Ly':Ly,'alpha':alpha,'Nx':Nx,'Ny':Ny,'tf':tf,'pt':pt}
        pickle.dump(dic,output,pickle.HIGHEST_PROTOCOL)
        
runsim(alpha,Nx,Ny,Lx,Ly)